In [12]:
from azureml.core import Workspace, Experiment, Environment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")
my_env = Environment.get(workspace=ws, name="AzureML-Scikit-learn-0.20.3")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')
print('Garima')
print('Garima')
run = exp.start_logging()

Workspace name: quick-starts-ws-136539
Azure region: southcentralus
Subscription id: 3d1a56d2-7c81-4118-9790-f85d1acf0c77
Resource group: aml-quickstarts-136539
Garima
Garima


In [13]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster-4"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [14]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.core.script_run_config import ScriptRunConfig

import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--batch-size': choice(25, 50, 100),
        '--first-layer-neurons': choice(10, 50, 200, 300, 500),
        '--second-layer-neurons': choice(10, 50, 200, 500),
        '--learning-rate': uniform(-6, -1)
    }
)

# Specify a Policy
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create an estimator for use with train.py
est = ScriptRunConfig(source_directory='./',script='train.py',compute_target=compute_target,environment=my_env)
### YOUR CODE HERE ###
### Set compute target skip this if you are running on your local computer script_run_config.run_config.target = compute_target ###
hyperdrive_config = HyperDriveConfig(run_config=est, 
                             hyperparameter_sampling=ps,
                             policy=early_termination_policy,
                             primary_metric_name='validation_acc', 
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                             max_total_runs=4,
                             max_concurrent_runs=4)

In [22]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
from azureml.widgets import RunDetails
# start the HyperDrive run

hyperdrive_run = exp.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_e42db101-1e4f-43dc-b293-12701fb851ef
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_e42db101-1e4f-43dc-b293-12701fb851ef?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-136539/workspaces/quick-starts-ws-136539

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-30T01:29:20.095758][API][INFO]Experiment created<END>\n"<START>[2021-01-30T01:29:20.8638996Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-01-30T01:29:20.772581][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-01-30T01:29:21.078008][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_e42db101-1e4f-43dc-b293-12701fb851ef
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_e42db101-1e4f-43dc-b293-12701fb851ef?wsid=/subscriptions/3d1a5

{'runId': 'HD_e42db101-1e4f-43dc-b293-12701fb851ef',
 'target': 'cpu-cluster-4',
 'status': 'Canceled',
 'startTimeUtc': '2021-01-30T01:29:19.569884Z',
 'endTimeUtc': '2021-01-30T01:37:08.323803Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'primary_metric_config': '{"name": "

In [25]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE .get_best_run_by_primary_metric() method ###

best_run = hyperdrive_run.get_best_run_by_primary_metric()
#best_run_metrics = best_run.get_details()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run.Id)
print('\n Accuracy: ', best_run_metrics['accuracy'])
print('\n Regularisation rate: ', parameter_values[1])
print('\n learning rate: ', parameter_values[3])
print('\n keep probability: ', parameter_values[5])
print('\n batch size: ', parameter_values[7])

best_run.download_file("/outputs/model.joblib", "hyperdrive_best_model.joblib")

AttributeError: 'NoneType' object has no attribute 'get_details'

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###